# Storing skymaps

## Read raw skymap files

In [1]:
import pickle
from pathlib import Path
import math
import numpy as np
from lsst.sphgeom import Box, ConvexPolygon, LonLat, UnitVector3d
import yaml

In [2]:
home_dir = "/sdf/home/o/olynn/"
raw_skymaps_dir = Path(home_dir, "skymap-to-poly-coords", "tests", "data", "raw-skymaps")

skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"
with open(skymap_path, "rb") as f:
    lsst_skymap = pickle.load(f)
lsst_skymap

## Utility function for later

In [3]:
def box_to_convex_polygon(box: Box) -> ConvexPolygon:
    if box.isEmpty():
        raise ValueError("Cannot convert an empty Box to a ConvexPolygon.")

    # Get the corners of the box
    lon_a, lon_b = box.getLon().getA().asRadians(), box.getLon().getB().asRadians()
    lon_min = min(lon_a, lon_b)
    lon_max = max(lon_a, lon_b)
    lat_a, lat_b = box.getLat().getA().asRadians(), box.getLat().getB().asRadians()
    lat_min = min(lat_a, lat_b)
    lat_max = max(lat_a, lat_b)
    # todo : this may be an improper assumption, considering RA wrap around!!

    bottom_left = LonLat.fromRadians(lon_min, lat_min)
    bottom_right = LonLat.fromRadians(lon_max, lat_min)
    top_right = LonLat.fromRadians(lon_max, lat_max)
    top_left = LonLat.fromRadians(lon_min, lat_max)

    # Convert corners to UnitVector3d
    vertices = [
        UnitVector3d(bottom_left),
        UnitVector3d(bottom_right),
        UnitVector3d(top_right),
        UnitVector3d(top_left),
    ]

    # Create and return the ConvexPolygon
    return ConvexPolygon(vertices)


## A note on rings sky map pixelization 
*(in html comment in this cell)*
<!---## Rings sky map pixelization
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.--->

## Storage options


We would like to support both inner and outer polygons for the tracts.  

Tracts are arranged in "rings", which span horizontal regions of the sky.

🔹 Inner polys:
- represent the exact boundaries of a given tract
- do not overlap

🔹 Outer polys:
- may (will?) overlap with adjacent tracts' outer polys
- are akin to margins in HATS catalogs

## Option 1: Two YAML files (inner polys and outer polys) that store each tracts' corners

#### Inner polys:
- Store per-tract: `ra_min`, `ra_max`, `dec_min`, `dec_max`
- Comes directly from `getRaDecRange(tract_index)`
- Requires 4 floats per tract, so for ~19k tracts:
  4 × 8 B × 19,000 ≈ ~600 KB (before YAML overhead; ~1–1.5 MB total)
- However, could compress to a `.npz` of around ~0.5–0.8 MB

#### Outer polys:
- todo

#### ✅ Pros
- Easy to understand and verify
- Doesn’t require any LSST WCS machinery
- Reading is fast + LSST-free

#### ❌ Cons
- Slightly more disk usage than an ultra-compressed model
- Less precise because of floating points?

### Write inner_poly and outer_poly files

In [9]:
# ##
# import lsst.geom as geom

# for tract_info in lsst_skymap:
#     print("\ntract_info" )
#     print(tract_info)
#     patch_vertices = [
#         [tract_info.getWcs().pixelToSky(geom.Point2D(corner)) for corner in patch.getOuterBBox().getCorners()]
#             for patch in tract_info
#         ]
#     for item in patch_vertices:
#         print("patch vertices:")
#         for vertex in item:
#             print(f"  {vertex}")
#     break


In [5]:
# import yaml
# from lsst.sphgeom import UnitVector3d

# def write_polygons(skymap, output_path, inner=True):
#     """Write exact inner polygons for each tract using 3D unit vectors.

#     Parameters
#     ----------
#     skymap : RingsSkyMap
#         The LSST SkyMap object.
#     output_path : str
#         Path to output YAML file.
#     inner : bool
#         If True, write inner polygons; if False, write outer polygons.
#         Default is True.
#     """
#     out = {"tracts": {}}

#     for tract in skymap:
#         tract_id = tract.getId()
#         if inner:
#             poly = tract.inner_sky_region
#             if isinstance(poly, Box):
#                 poly = box_to_convex_polygon(poly)
#         else:
#             poly = tract.outer_sky_polygon

#         out["tracts"][tract_id] = [
#             [v[0], v[1], v[2]] for v in poly.getVertices()
#         ]

#     with open(output_path, "w") as f:
#         yaml.dump(out, f, sort_keys=False)

#     polygon_type = "Inner" if inner else "Outer"
#     print(f"✅ {polygon_type} polygons written to {output_path}")


In [12]:
import yaml
import lsst.geom as geom
from lsst.sphgeom import LonLat, Box

def write_polygons_ra_dec(skymap, output_path, inner=True, write_patches=False):
    """Write exact inner or outer polygons for each tract using RA/Dec coordinates.

    Optionally includes patch boundaries as well.

    Parameters
    ----------
    skymap : RingsSkyMap
        The LSST SkyMap object.
    output_path : str
        Path to output YAML file.
    inner : bool
        If True, write inner polygons; if False, write outer polygons.
    write_patches : bool, optional
        If True, also include patch polygon vertices. Default is False.
    """
    out = {"tracts": {}}

    for tract in skymap:
        tract_id = tract.getId()
        if inner:
            poly = tract.inner_sky_region
            if isinstance(poly, Box):
                poly = box_to_convex_polygon(poly)
        else:
            poly = tract.outer_sky_polygon

        # Tract polygon in RA/Dec
        ra_dec_vertices = []
        for vec in poly.getVertices():
            lonlat = LonLat(vec)
            ra = lonlat.getLon().asDegrees() % 360.0
            dec = lonlat.getLat().asDegrees()
            ra_dec_vertices.append([ra, dec])

        out["tracts"][tract_id] = {"polygon": ra_dec_vertices}

        # Optional: include patch polygons
        if write_patches:
            patch_polys = []
            for patch in tract:
                corners = patch.getOuterBBox().getCorners()
                wcs = tract.getWcs()
                patch_ra_dec = []
                for corner in corners:
                    sky_pt = wcs.pixelToSky(geom.Point2D(corner))
                    ra = sky_pt.getLongitude().asDegrees() % 360.0
                    dec = sky_pt.getLatitude().asDegrees()
                    patch_ra_dec.append([ra, dec])
                patch_polys.append(patch_ra_dec)

            out["tracts"][tract_id]["patches"] = patch_polys

    with open(output_path, "w") as f:
        yaml.dump(out, f, sort_keys=False)

    print(f"✅ {'Inner' if inner else 'Outer'} polygons written to {output_path} (RA/Dec format)")
    if write_patches:
        print("🧩 Patch polygons included.")


In [13]:
skymap_out_dir = "/sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/"

inner_poly_path = Path(skymap_out_dir) / "inner_polys_with_patches.yaml"
write_polygons_ra_dec(lsst_skymap, inner_poly_path, inner=True, write_patches=True)

outer_poly_path = Path(skymap_out_dir) / "outer_polys.yaml"
write_polygons_ra_dec(lsst_skymap, outer_poly_path, inner=False, write_patches=False)

✅ Inner polygons written to /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/inner_polys_with_patches.yaml (RA/Dec format)
🧩 Patch polygons included.
✅ Outer polygons written to /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/outer_polys_with_patches.yaml (RA/Dec format)
🧩 Patch polygons included.


## Option 1B: Recording RA/dec rather than UnitVectors

Can we get radec to play well with the lsst.geom interfaces? There's some support for LonLat...

todo : description and code

## Option 2: A hybrid approach, because in theory, we can assume a shared min/max declination across each ring?

todo : description and code

## Option 3: Reconstruct everything using projection + geometry

What you'd need to store:  

- Global:
  - `projection: "TAN"`
  - `pixel_scale_deg`: (or arcsec/pixel)
  - `tract_width`, `tract_height`: in pixels
  - `overlap_deg`: or overlap in pixels (need this for inner bounds)

- Per-tract:
  - `tract_id`
  - `center_ra`, `center_dec`
  - `rotation` (optional, usually 0)
  - `flipX` (optional, for WCS handedness)

Then, you reconstruct inner and outer polys by:
- Creating a fake `SkyWcs` from these params
- Converting pixel corners of `[0,0]` → `[width,height]` to sky coordinates
- Optionally applying overlap trimming in pixel space

✅ Pros
- Super compact: ~2 floats + a few constants per tract
- Elegant reuse of LSST projection machinery
- One unified format

❌ Cons
- Requires LSST-style WCS math to reconstruct accurately
- Introduces fragility: assumptions about projection type and tract uniformity
- Slightly harder to read/debug